In [3]:
import pandas as pd

data_A = pd.read_csv('data/mid_nm.csv', sep=",")
df_A = pd.DataFrame(data_A, columns=data_A.keys())

data_B = pd.read_csv('data/mitm.csv', sep=",")
df_B = pd.DataFrame(data_B, columns=data_B.keys())

data_C = pd.read_csv('data/modbus_Query_Flooding.csv', sep=",")
df_C = pd.DataFrame(data_C, columns=data_C.keys())

data_D = pd.read_csv('data/normal_file.csv', sep=",")
df_D = pd.DataFrame(data_D, columns=data_D.keys())

data_E = pd.read_csv('data/pingFloodDDoS.csv', sep=",")
df_E = pd.DataFrame(data_E, columns=data_E.keys())

data_F = pd.read_csv('data/tcpSYNFloodDDoS.csv', sep=",")
df_F = pd.DataFrame(data_F, columns=data_F.keys())

res = pd.concat([df_A, df_B, df_C, df_D, df_E,df_F])

res.to_csv("full.csv")

In [4]:
import pandas as pd
import numpy as np

full_data = pd.read_csv("full.csv", sep=",")
full_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_full = pd.DataFrame(full_data, columns=full_data.keys())

df_shuffed = df_full.iloc[np.random.permutation(df_full.index)].reset_index(drop=True)

full_len = len(df_shuffed)
train_len = int(full_len * 0.6) 

train_df = df_shuffed.iloc[:train_len,:]
test_df = df_shuffed.iloc[train_len:,:]

print("train data: " + str(len(train_df)))
print("test data: " + str(len(test_df)))

train_df.to_csv("train.csv")
test_df.to_csv("test.csv")


train data: 2474127
test data: 1649419


In [6]:
import pandas as pd

train_data = pd.read_csv('train.csv', sep=",")
test_data = pd.read_csv('test.csv', sep=",")

train_data = pd.DataFrame(train_data, columns=train_data.keys())
test_df = pd.DataFrame(test_data, columns=test_data.keys())

print("train data: " + str(len(train_df)))
print("test data: " + str(len(test_df)))

print(test_data.keys()[28:])

train data: 2474127
test data: 1649419
Index(['ICMP code_7bit', 'n_same_src_ip', 'n_same_dst_ip',
       'n_same_src_ip_port', 'n_same_dst_ip_port', 'n_dst_ip_src_ip_count',
       'n_src_dst_ip_src_port_count', 'n_src_dst_ip_dst_port_count',
       'n_src_ip_data_len', 'n_dst_ip_data_len', 'n_src_ip_ICMP_count',
       'n_src_ip_SYN_count', 'n_dst_ip_ICMP_count', 'n_dst_ip_SYN_count',
       'attack_type'],
      dtype='object')


In [1]:
import pandas as pd
import numpy as np
import time

class Data_Load:
    def __init__(self, test_file_loc, train_file_loc, extract_list, Y_loc=41):
        self.extract_list = extract_list
        test_data = pd.read_csv(test_file_loc, sep=",")
        test_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        test_data = test_data.iloc[np.random.permutation(test_data.index)].reset_index(drop=True)
        train_data = pd.read_csv(train_file_loc, sep=",")
        train_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
        train_data = train_data.iloc[np.random.permutation(train_data.index)].reset_index(drop=True)
        
        self.X_test = test_data.iloc[:, extract_list]
        self.X_train = train_data.iloc[:, extract_list]
        self.Y_test = test_data.iloc[:, [Y_loc]]
        self.Y_train = train_data.iloc[:, [Y_loc]]
        
        self.input_len = len(extract_list)
        
l1 = [] #full data
for i in range(41):
    l1.append(i)
l2 = [0, 1]+ l1[28:] #no flag bits
l3 = l1[28:] #only relative info
l4 = l1[:34] + [36] + [39, 40] # sub some same src_ip info
'''
l4:
n초간 동일 src_ip에서 보낸 data 길이 총합(bytes)
n초간 동일 src_ip에서 보낸 ICMP 패킷 수
n초간 동일 src_ip에서 보낸 SYN 패킷 수

제거
'''
    
data1 = Data_Load("test.csv", "train.csv", l1)

data2 = Data_Load("test.csv", "train.csv", l2)

data3 = Data_Load("test.csv", "train.csv", l3)

data4 = Data_Load("test.csv", "train.csv", l4)

In [2]:
from sklearn.svm import SVC
import time
from joblib import dump, load
from sklearn.metrics import precision_score, recall_score, f1_score

class SVM_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        self.model = SVC(kernel='rbf', C=8, gamma=0.1, max_iter=500)
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= Y_train.to_numpy().flatten()
        self.Y_test= Y_test.to_numpy().flatten()
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        self.f1 = 0
        self.prec = 0
        self.recall = 0
        
    def train(self):
        train_time_start = time.time()
        self.model.fit(self.X_train, self.Y_train)
        train_time_end = time.time()
        
        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        y_pred  = self.model.predict(self.X_test)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = np.mean(y_pred == self.Y_test)
        self.f1 = f1_score(self.Y_test, y_pred, average='micro')
        self.prec = precision_score(self.Y_test, y_pred, average='micro')
        self.recall = recall_score(self.Y_test, y_pred, average='micro')
        print("prediction accuracy: {:.2f}".format(self.acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('SVM_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        dump(self.model, 'SVM_Model'+tm_string+'.pkl')
        
svm1 = SVM_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
svm1.train()
svm1.test()
svm1.save()

svm2 = SVM_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
svm2.train()
svm2.test()
svm2.save()

svm3 = SVM_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
svm3.train()
svm3.test()
svm3.save()

svm4 = SVM_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
svm4.train()
svm4.test()
svm4.save()

C:\Users\NEXON\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


prediction accuracy: 0.82


C:\Users\NEXON\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


prediction accuracy: 0.88


C:\Users\NEXON\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


prediction accuracy: 0.81


C:\Users\NEXON\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


prediction accuracy: 0.78


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import time
from joblib import dump, load
from sklearn.metrics import precision_score, recall_score, f1_score

class K_NN_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        #weights: "uniform" or "distance"
        self.model = KNeighborsClassifier(n_neighbors = 5, weights = "distance")
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= Y_train.to_numpy().flatten()
        self.Y_test= Y_test.to_numpy().flatten()
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        self.f1 = 0
        self.prec = 0
        self.recall = 0
        
    def train(self):
        train_time_start = time.time()
        self.model.fit(self.X_train, self.Y_train)
        train_time_end = time.time()
        
        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        y_pred  = self.model.predict(self.X_test)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = np.mean(y_pred == self.Y_test)
        self.f1 = f1_score(self.Y_test, y_pred, average='micro')
        self.prec = precision_score(self.Y_test, y_pred, average='micro')
        self.recall = recall_score(self.Y_test, y_pred, average='micro')
        print("prediction accuracy: {:.2f}".format(self.acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('K_NN_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        dump(self.model, 'K_NN_Model'+tm_string+'.pkl')
        
        
knn1 = K_NN_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
knn1.train()
#knn1.test()
knn1.save()

knn2 = K_NN_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
knn2.train()
#knn2.test()
knn2.save()

knn3 = K_NN_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
knn3.train()
#knn3.test()
knn3.save()

knn4 = K_NN_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
knn4.train()
#knn4.test()
knn4.save()

In [3]:
from sklearn.ensemble import RandomForestClassifier
import time
from joblib import dump, load
from sklearn.metrics import precision_score, recall_score, f1_score

class Random_Forest_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        self.model = RandomForestClassifier(n_estimators=50)
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= Y_train.to_numpy().flatten()
        self.Y_test= Y_test.to_numpy().flatten()
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        self.f1 = 0
        self.prec = 0
        self.recall = 0
        
    def train(self):
        train_time_start = time.time()
        self.model.fit(self.X_train, self.Y_train)
        train_time_end = time.time()
        
        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        y_pred  = self.model.predict(self.X_test)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = np.mean(y_pred == self.Y_test)
        self.f1 = f1_score(self.Y_test, y_pred, average='micro')
        self.prec = precision_score(self.Y_test, y_pred, average='micro')
        self.recall = recall_score(self.Y_test, y_pred, average='micro')
        print("prediction accuracy: {:.2f}".format(self.acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('Random_Forest_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        dump(self.model, 'Random_Forest_Model'+tm_string+'.pkl')


rfm1 = Random_Forest_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
rfm1.train()
rfm1.test()
rfm1.save()

rfm2 = Random_Forest_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
rfm2.train()
rfm2.test()
rfm2.save()

rfm3 = Random_Forest_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
rfm3.train()
rfm3.test()
rfm3.save()

rfm4 = Random_Forest_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
rfm4.train()
rfm4.test()
rfm4.save()

prediction accuracy: 0.99
prediction accuracy: 0.99
prediction accuracy: 0.99
prediction accuracy: 0.99


In [4]:
from sklearn.cluster import KMeans
import time
from joblib import dump, load
from sklearn.metrics import precision_score, recall_score, f1_score

class K_Means_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        self.model = KMeans(n_clusters=5)
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= Y_train.to_numpy().flatten() #통일성을 위해 있는 더미
        self.Y_test= Y_test.to_numpy().flatten()
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        self.f1 = 0
        self.prec = 0
        self.recall = 0
        
    def train(self):
        train_time_start = time.time()
        self.model.fit(self.X_train)
        train_time_end = time.time()
        
        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        y_pred  = self.model.predict(self.X_test)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = np.mean(y_pred == self.Y_test)
        self.f1 = f1_score(self.Y_test, y_pred, average='micro')
        self.prec = precision_score(self.Y_test, y_pred, average='micro')
        self.recall = recall_score(self.Y_test, y_pred, average='micro')
        print("prediction accuracy: {:.2f}".format(self.acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('K_Means_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        dump(self.model, 'K_Means_Model'+tm_string+'.pkl')
        
kmm1 = K_Means_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
kmm1.train()
kmm1.test()
kmm1.save()

kmm2 = K_Means_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
kmm2.train()
kmm2.test()
kmm2.save()

kmm3 = K_Means_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
kmm3.train()
kmm3.test()
kmm3.save()

kmm4 = K_Means_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
kmm4.train()
kmm4.test()
kmm4.save()

prediction accuracy: 0.58
prediction accuracy: 0.03
prediction accuracy: 0.01
prediction accuracy: 0.17


In [8]:
from sklearn import tree
import time
from joblib import dump, load
from sklearn.metrics import precision_score, recall_score, f1_score

class DT_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        self.model = tree.DecisionTreeClassifier()
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= Y_train.to_numpy().flatten()
        self.Y_test= Y_test.to_numpy().flatten()
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        self.f1 = 0
        self.prec = 0
        self.recall = 0
        
    def train(self):
        train_time_start = time.time()
        self.model.fit(self.X_train, self.Y_train)
        train_time_end = time.time()

        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        y_pred  = self.model.predict(self.X_test)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = np.mean(y_pred == self.Y_test)
        self.f1 = f1_score(self.Y_test, y_pred, average='micro')
        self.prec = precision_score(self.Y_test, y_pred, average='micro')
        self.recall = recall_score(self.Y_test, y_pred, average='micro')
        print("prediction accuracy: {:.2f}".format(self.acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('DT_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        dump(self.model, 'DT_Model'+tm_string+'.pkl')
        
dtm1 = DT_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
dtm1.train()
dtm1.test()
dtm1.save()

dtm2 = DT_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
dtm2.train()
dtm2.test()
dtm2.save()

dtm3 = DT_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
dtm3.train()
dtm3.test()
dtm3.save()

dtm4 = DT_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
dtm4.train()
dtm4.test()
dtm4.save()

prediction accuracy: 0.99
prediction accuracy: 0.99
prediction accuracy: 0.99
prediction accuracy: 0.99


In [2]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

class DNN_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= tf.keras.utils.to_categorical(Y_train.to_numpy().flatten())
        self.Y_test= tf.keras.utils.to_categorical(Y_test.to_numpy().flatten())
        
        #need to change shape
        self.model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=[self.input_len]),
            layers.Dense(48, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(16, activation='relu'),
            layers.Dense(5, activation='softmax')
        ])
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy',f1_m,precision_m, recall_m])
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        self.f1 = 0
        self.prec = 0
        self.recall = 0
        
    def train(self):
        train_time_start = time.time()
        self.model.fit(self.X_train,
                       self.Y_train,  
                       epochs=5)
        train_time_end = time.time()
        
        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        test_loss, test_acc, test_f1, test_prec, test_recall = self.model.evaluate(self.X_test, self.Y_test, verbose=2)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = test_acc
        self.f1 = test_f1
        self.prec = test_prec
        self.recall = test_recall
        print("prediction accuracy: {:.2f}".format(test_acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('DNN_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        self.model.save("DNN_Model"+tm_string)
        
        
dnn1 = DNN_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
dnn1.train()
dnn1.test()
dnn1.save()

dnn2 = DNN_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
dnn2.train()
dnn2.test()
dnn2.save()

dnn3 = DNN_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
dnn3.train()
dnn3.test()
dnn3.save()

dnn4 = DNN_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
dnn4.train()
dnn4.test()
dnn4.save()

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import time

class Autoencoder(Model):
    def __init__(self, input_len):
        self.input_len = input_len
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(32, activation='relu', input_shape=[self.input_len]),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(16, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(self.input_len, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
class Encoder_Model:
    def __init__(self, input_len, X_train, X_test, Y_train, Y_test, extract_list):
        self.extract_list = extract_list
        self.input_len = input_len
        
        self.X_train= X_train.to_numpy()
        self.X_test= X_test.to_numpy()
        self.Y_train= tf.keras.utils.to_categorical(Y_train.to_numpy().flatten())
        self.Y_test= tf.keras.utils.to_categorical(Y_test.to_numpy().flatten())
        
        self.autoencoder = Autoencoder(self.input_len)
        self.model = None
        
        self.autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        
        self.train_time = 0
        self.test_time = 0
        self.acc = 0
        
    def train(self):
        train_time_start = time.time()
        self.autoencoder.fit(self.X_train,
                             self.X_train,  
                             validation_data=(self.X_test, self.X_test),
                             epochs=5)
        
        self.model = tf.keras.Sequential([
            self.autoencoder.encoder,
            layers.Dense(8, activation='relu'),
            layers.Dense(5, activation='softmax')
        ])
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy',f1_m,precision_m, recall_m])
        
        self.model.fit(self.X_train,
                       self.Y_train,  
                       epochs=5)
        
        train_time_end = time.time()
        
        self.train_time = train_time_end - train_time_start
        
    def test(self):
        test_time_start = time.time()
        test_loss, test_acc, test_f1, test_prec, test_recall = self.model.evaluate(self.X_test, self.Y_test, verbose=2)
        test_time_end = time.time()
        
        self.test_time = test_time_end - test_time_start
        self.acc = test_acc
        self.f1 = test_f1
        self.prec = test_prec
        self.recall = test_recall
        print("prediction accuracy: {:.2f}".format(test_acc)) # 예측 정확도
        
    def save(self):
        tm = time.localtime()
        tm_string = time.strftime('%Y-%m-%d-%H_%M_%S', tm)
        f = open('Encoder_Model'+tm_string+'.txt', 'w')
        f.write('train time: ' + str(self.train_time)+'\n')
        f.write('test time: ' + str(self.test_time)+'\n')
        f.write('acc: ' + str(self.acc)+'\n')
        f.write('f1: ' + str(self.f1)+'\n')
        f.write('prec: ' + str(self.prec)+'\n')
        f.write('recall: ' + str(self.recall)+'\n')
        f.write('extract_list: ' + str(self.extract_list)+'\n')
        
        self.model.save("Encoder_Model"+tm_string)
        
encode1 = Encoder_Model(data1.input_len, data1.X_train, data1.X_test, data1.Y_train, data1.Y_test, data1.extract_list)
encode1.train()
encode1.test()
encode1.save()

encode2 = Encoder_Model(data2.input_len, data2.X_train, data2.X_test, data2.Y_train, data2.Y_test, data2.extract_list)
encode2.train()
encode2.test()
encode2.save()

encode3 = Encoder_Model(data3.input_len, data3.X_train, data3.X_test, data3.Y_train, data3.Y_test, data3.extract_list)
encode3.train()
encode3.test()
encode3.save()

encode4 = Encoder_Model(data4.input_len, data4.X_train, data4.X_test, data4.Y_train, data4.Y_test, data4.extract_list)
encode4.train()
encode4.test()
encode4.save()

Epoch 1/5
77317/77317 [==============================] - 54s 693us/step - loss: 0.4848 - val_loss: 0.4840
Epoch 2/5
77317/77317 [==============================] - 54s 703us/step - loss: 0.4833 - val_loss: 0.4839
Epoch 3/5
77317/77317 [==============================] - 56s 719us/step - loss: 0.4824 - val_loss: 0.4828
Epoch 4/5
77317/77317 [==============================] - 54s 700us/step - loss: 0.4821 - val_loss: 0.4826
Epoch 5/5
77317/77317 [==============================] - 57s 734us/step - loss: 0.4821 - val_loss: 0.4826
Epoch 1/5
77317/77317 [==============================] - 45s 567us/step - loss: 0.0485 - accuracy: 0.9853 - f1_m: 0.9844 - precision_m: 0.9856 - recall_m: 0.9835
Epoch 2/5
77317/77317 [==============================] - 44s 568us/step - loss: 0.0410 - accuracy: 0.9865 - f1_m: 0.9865 - precision_m: 0.9865 - recall_m: 0.9865
Epoch 3/5
77317/77317 [==============================] - 42s 547us/step - loss: 0.0408 - accuracy: 0.9865 - f1_m: 0.9865 - precision_m: 0.9865 - r